# Kneser-Ney smoothingによるn-gram言語モデルを使った句点挿入(生文デモ)
##  [KenLM](https://github.com/kpu/kenlm) で学習済みのモデル(*.klm)を読み込み

In [1]:
import kenlm
LM = 'data/model.100K.klm'
model = kenlm.Model(LM)
print('{0}-gram model'.format(model.order))

3-gram model


## 入力を分かち書き

In [2]:
import MeCab


class WordDivider:

    def __init__(self, dictionary=".mecabrc", is_normalize=False):
        self.dictionary = dictionary
        self.tagger = MeCab.Tagger(self.dictionary)
        self.tagger.parse("") # necessary in Python3
        self.is_normalize = is_normalize

    def extract_words(self, text, with_pos=True):
        if not text:
            return []

        words = []
        # normalize text before MeCab processing
        if self.is_normalize:
            text = text.lower()

        node = self.tagger.parseToNode(text)
        while node:
            features = node.feature.split(',')
            # extract word surface (configurable)
            word_to_append = node.surface
            if with_pos:
                pos = features[0]
                word_to_append = '{}/{}'.format(word_to_append, pos)
            words.append(word_to_append)
            node = node.next

        return words

def wakati(line):
    wd = WordDivider(is_normalize=True)
    words = wd.extract_words(line, with_pos=False)
    return ' '.join(filter(None, words))

# 句読点挿入モデル
## 各位置で句点を挿入する/しないで文全体の尤度比較し、尤度が上がれば句点挿入

In [3]:
PUNKTS = ['、', '。']

# windowはngramのnにあわせる
def punkt_insert(sentence, punkt='。', window=3, debug=False):
    words = sentence.split(' ')
    pnkt_pos = []
    for i in range(len(words)-window+1):
        prob = model.score(' '.join(words))
        prob_p = model.score(' '.join(words[:i+window] + [punkt] + words[i+window:]))
        if prob_p > prob:
            if debug:
                print(prob_p, prob, words[i:i+window])
            pnkt_pos.append(i+window)
    return pnkt_pos

def punkt_decode(sentence, punkt_pos, punkt='。'):
    words = sentence.split(' ')
    pnkt_sent = []
    for i in range(len(words)):
        pnkt_sent.append(words[i])
        if i+1 in punkt_pos:
            pnkt_sent.append(punkt)
    return ' '.join(pnkt_sent)

# 句読点付き文から句点挿入位置を抽出
def extract_positions(sentence, punkt='。'):
    true_positions = []
    c_pos = 0
    for w in sentence.split(' '):
        if w in PUNKTS:
            if w == punkt:
                true_positions.append(c_pos)
        else:
            c_pos += 1
    return true_positions

# 句読点無し文に対する句点挿入位置の予測結果から精度と再現率を計算
def prf(punkt_pred, punkt_true):
    tp = set.intersection(set(punkt_pred), set(punkt_true))
    recall = len(tp) / len(punkt_true) if len(punkt_true) > 0 else 0
    precision = len(tp) / len(punkt_pred) if len(punkt_pred) > 0 else 0
    fval = 2 / (1/precision + 1/recall) if precision and recall else 0.
    return precision, recall, fval

## 句読点なし文に句読点を挿入

In [4]:
sentence_true = '今日も、いい天気だ。明日の天気も晴れらしい。'
sentence_true = wakati(sentence_true)
print('入力文:')
print(sentence_true)
sentence_raw = ''.join(filter(lambda x: x not in PUNKTS, sentence_true))

入力文:
今日 も 、 いい 天気 だ 。 明日 の 天気 も 晴れ らしい 。


### 句読点付き正解文と比較し、精度・再現率・F値を算出

In [5]:
sentence_wakati = wakati(sentence_raw)
punkt_position = punkt_insert(sentence_wakati)
sentence_pred = punkt_decode(sentence_wakati, punkt_position)
print('予測結果:')
print(sentence_pred)
print(''.join(sentence_pred.split(' ')))
print(extract_positions(sentence_pred), extract_positions(sentence_true))
p,r,f = prf(extract_positions(sentence_pred), extract_positions(sentence_true))
print('Precision:{0:.5g}, Recall:{1:.5g}, F-value:{2:.5g}'.format(p,r,f))

予測結果:
今日 も いい 天気 だ 。 明日 の 天気 も 晴れ らしい 。
今日もいい天気だ。明日の天気も晴れらしい。
[5, 11] [5, 11]
Precision:1, Recall:1, F-value:1
